In [1]:
!pip install ktrain==0.26.2
import ktrain
from ktrain import text
import pandas as pd

     |████████████████████████████████| 25.3MB 119kB/s 
     |████████████████████████████████| 6.8MB 16.3MB/s 
     |████████████████████████████████| 983kB 25.7MB/s 
     |████████████████████████████████| 266kB 36.3MB/s 
     |████████████████████████████████| 1.9MB 19.5MB/s 
     |████████████████████████████████| 1.2MB 31.8MB/s 
     |████████████████████████████████| 471kB 25.4MB/s 
     |████████████████████████████████| 3.3MB 25.1MB/s 
     |████████████████████████████████| 901kB 26.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.2-cp37-none-any.whl size=25277794 sha256=32d5b16f194b0ebd625b8aff7eff45cfe982a0da0288a6bd1f25782ea3918ed0
  Stored in directory: /root/.cache/pip/wheels/67/2e/f1/c72afa08df8b2d984b910dea228902ce81dae4511afe9fafd2
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=d553161429b8e3eed12e171e08264c21a1a5894301f8cc4eec02cccd73041258
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [2]:
# Downloading and loading the dataset
!gdown https://drive.google.com/uc?id=1-8urBLVtFuuvAVHi0s000e7r0KPUgt9f

Downloading...
From: https://drive.google.com/uc?id=1-8urBLVtFuuvAVHi0s000e7r0KPUgt9f
To: /content/reviews_Digital_Music_5.json
89.0MB [00:01, 54.8MB/s]


In [3]:
df = pd.read_json(r'reviews_Digital_Music_5.json',lines=True)

In [4]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3EBHHCZO6V2A4,5555991584,"Amaranth ""music fan""","[3, 3]","It's hard to believe ""Memory of Trees"" came ou...",5,Enya's last great album,1158019200,"09 12, 2006"
1,AZPWAXJG9OJXV,5555991584,bethtexas,"[0, 0]","A clasically-styled and introverted album, Mem...",5,Enya at her most elegant,991526400,"06 3, 2001"
2,A38IRL0X2T4DPF,5555991584,bob turnley,"[2, 2]",I never thought Enya would reach the sublime h...,5,The best so far,1058140800,"07 14, 2003"
3,A22IK3I6U76GX0,5555991584,Calle,"[1, 1]",This is the third review of an irish album I w...,5,Ireland produces good music.,957312000,"05 3, 2000"
4,A1AISPOIIHTHXX,5555991584,"Cloud ""...""","[1, 1]","Enya, despite being a successful recording art...",4,4.5; music to dream to,1200528000,"01 17, 2008"


In [5]:
# Selecting only the rows needed
df = df[['reviewText','overall']]
df.head()

,reviewText,overall
0,"It's hard to believe ""Memory of Trees"" came ou...",5
1,"A clasically-styled and introverted album, Mem...",5
2,I never thought Enya would reach the sublime h...,5
3,This is the third review of an irish album I w...,5
4,"Enya, despite being a successful recording art...",4


In [7]:
# Mapping ratings to classes
# 1,2 and 3 as negativa whereas 4 and 5 as positive
sentiment = {1:'negative',2:'negative',3:'negative',
             4:'positive',5:'positive'}
df['sentiment'] = df['overall'].map(sentiment)

In [9]:
df = df[['reviewText','sentiment']]
df.head()

,reviewText,sentiment
0,"It's hard to believe ""Memory of Trees"" came ou...",positive
1,"A clasically-styled and introverted album, Mem...",positive
2,I never thought Enya would reach the sublime h...,positive
3,This is the third review of an irish album I w...,positive
4,"Enya, despite being a successful recording art...",positive


In [10]:
# Creating the train and test sets
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train_df = df, 
                                                                   text_column = 'reviewText',
                                                                   label_columns=['sentiment'],
                                                                   maxlen=100, 
                                                                   max_features=100000, # the maximum number of words used in vocabulary
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.1)

['negative', 'positive']
       negative  positive
16527       0.0       1.0
15051       0.0       1.0
61512       0.0       1.0
24645       0.0       1.0
26974       0.0       1.0
['negative', 'positive']
       negative  positive
53736       1.0       0.0
25091       0.0       1.0
31615       0.0       1.0
26483       0.0       1.0
56644       1.0       0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [11]:
# Defining the classifier model
model = text.text_classifier(name='bert', train_data = (x_train, y_train) , preproc=preproc, metrics=['accuracy'])

Is Multi-Label? False
maxlen is 100
done.


In [12]:
# Creating an instance learner to train the model
learner = ktrain.get_learner(model = model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=32, 
                             use_multiprocessing = True)

In [13]:
# Training the model using the fit_onecycle function
learner.fit_onecycle(lr=2e-5, epochs=1,checkpoint_folder='output')



begin training using onecycle policy with max lr of 2e-05...
1820/1820 [==============================] - 2725s 1s/step - loss: 0.3721 - accuracy: 0.8465 - val_loss: 0.2832 - val_accuracy: 0.8847


In [14]:
# Using the trained model to make predictions
predictor = ktrain.get_predictor(learner.model, preproc)

In [16]:
text = "I loved the song" # test text
predictor.predict(text)

'positive'

In [17]:
text = "I couldn't get the tickets for the concert" # test text
predictor.predict(text)

'negative'

In [18]:
text = "I hope one day the pandemic will come to an end" # test text
predictor.predict(text)

'positive'